In [ ]:
!pip install feature_engine
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# for google colab
from google.colab import drive
# mount your Google Drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# for google colab
# copy all files from "HW5" directory in Google drive to current directory
!cp -r ./gdrive/MyDrive/Final/* .

In [ ]:
import os
import sys
import joblib
import numpy as np
import pandas as pd
import gc; gc.enable()
from lightgbm import LGBMClassifier
from sklearn.impute import KNNImputer
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import GaussianNB
from feature_engine.encoding import WoEEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression, HuberRegressor
import warnings; warnings.filterwarnings("ignore")
from tensorflow.keras import Sequential  
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_addons as tfa

pd.options.display.max_columns = 999

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
tf.random.set_seed(42)

# Data Preprocessing

In [ ]:
def preprocessing(df_train, df_test):
  # dictionnary of dictionnaries (for the 11 best correlated measurement columns), 
  # we will use the dictionnaries below to select the best correlated columns according to the product code)
  # Only for 'measurement_17' we make a 'manual' selection :
  full_fill_dict ={}
  full_fill_dict['measurement_17'] = {
      'A': ['measurement_5','measurement_6','measurement_8'],
      'B': ['measurement_4','measurement_5','measurement_7'],
      'C': ['measurement_5','measurement_7','measurement_8','measurement_9'],
      'D': ['measurement_5','measurement_6','measurement_7','measurement_8'],
      'E': ['measurement_4','measurement_5','measurement_6','measurement_8'],
      'F': ['measurement_4','measurement_5','measurement_6','measurement_7'],
      'G': ['measurement_4','measurement_6','measurement_8','measurement_9'],
      'H': ['measurement_4','measurement_5','measurement_7','measurement_8','measurement_9'],
      'I': ['measurement_3','measurement_7','measurement_8']
  }

  # data = train + test => take both train and test data into consideration
  data = pd.concat([df_train, df_test]) 
  # construct additional column to record the loss data for measurement_3 & measurement_5 & area
  data['m3_missing'] = 1 * data['measurement_3'].isnull()
  data['m5_missing'] = 1 * data['measurement_5'].isnull()
  data['area'] = data['attribute_2'] * data['attribute_3']

  # calculate the important order of all measurements which depends on correlation
  # filter out the column that has no relation to measurement ramaining the related one and keep them in corelated_data
  correlated_data = data[['measurement_' + str(i) for i in range(18)] + ['failure', 'area']]
  val = []
  col =[]
  for x in range(3,17):
    cor_val = correlated_data.corr()['measurement_' + str(x)] # data.corr()表示了data中的两个变量之间的相关性
    cor_val = np.absolute(cor_val)
    total_val = np.sum(cor_val.sort_values(ascending=False)[1:4]) # get most 3 correlated value
    val.append(np.round(total_val,3)) 
    col.append('measurement_' + str(x))

  c = pd.DataFrame()
  c['corelated columns'] = col
  c['correlated value'] = val
  c = c.sort_values(by='correlated value', ascending=False).reset_index(drop=True)

  # we just pick the most important 10 measurements
  # find the best corelated columns based on the product code as the initial format of measurement17
  for i in range(10):
    measurement_col = 'measurement_' + c.iloc[i,0][12:] # we select the next best correlated column since the first one is initially set-up measurement17 
    fill_dict = {}
    for x in data['product_code'].unique() : 
      cor_val = correlated_data[data['product_code'] == x].corr()[measurement_col]
      cor_val = np.absolute(cor_val).sort_values(ascending=False)
      measurement_col_dic = {}
      measurement_col_dic[measurement_col] = cor_val[1:5].index.tolist() # keep the most important 4 measurement
      fill_dict[x] = measurement_col_dic[measurement_col]
    full_fill_dict[measurement_col] = fill_dict

  # start running depends on product code
  for code in data['product_code'].unique():
    # use HuberRegressor to fill the missing value
    for measurement_col in list(full_fill_dict.keys()):
      # extract the current product code data
      tmp = data[data['product_code'] == code]
      # extract the correlated measurement we just claculated
      column = full_fill_dict[measurement_col][code]
      # collect all corelated measurement's data and drop rows which contain missing values
      tmp_train = tmp[column + [measurement_col]].dropna(how='any')
      # collect the data that doesn't miss data
      tmp_test = tmp[(tmp[column].isnull().sum(axis=1) == 0) & (tmp[measurement_col].isnull())]
      model = HuberRegressor(epsilon=1.9)
      model.fit(tmp_train[column], tmp_train[measurement_col])
      data.loc[(data['product_code'] == code) & (data[column].isnull().sum(axis=1) == 0) & (data[measurement_col].isnull()), measurement_col] = model.predict(tmp_test[column])

    # use KNNImputer to fill the missing value
    nullValue_cols = [col for col in df_train.columns if df_train[col].isnull().any()] # keep the column with loss data
    # calculate the total missing data depends on each measurement and current product code
    NA = data.loc[data['product_code'] == code, nullValue_cols].isnull().sum().sum()
    # Imputation for completing missing values using k-Nearest Neighbors.
    model1 = KNNImputer(n_neighbors=3) 
    feature = ['loading'] + ['measurement_' + str(i) for i in range(18)]
    data.loc[data['product_code'] == code, feature] = model1.fit_transform(data.loc[data['product_code'] == code, feature])

  # measurement 3 - 16 looks like they belong to the same group
  data['measurement_avg'] = data[['measurement_' + str(i) for i in range(3, 17)]].mean(axis=1)
  
  # replaces categories by the weight of evidence
  df_train = data.iloc[:len(df_train),:]
  woe_encoder = WoEEncoder(variables=['attribute_0'])
  woe_encoder.fit(df_train, df_train['failure'])
  df_train = woe_encoder.transform(df_train)
  
  return df_train

df_train = preprocessing(train, test)

Columns selected by correlation sum of the 3 first rows : 


,Selected columns,correlation total
0,measurement_8,0.454
1,measurement_11,0.395
2,measurement_5,0.386
3,measurement_6,0.365
4,measurement_7,0.336
5,measurement_4,0.331
6,measurement_15,0.301
7,measurement_10,0.300
8,measurement_16,0.252
9,measurement_14,0.225



-------- Product code A ----------

filled by linear model :
measurement_17 : 386
measurement_8 : 167
measurement_11 : 225
measurement_5 : 113
measurement_6 : 146
measurement_7 : 153
measurement_4 : 79
measurement_15 : 273
measurement_10 : 209
measurement_16 : 293
measurement_14 : 237

2281 filled by linear model 
1568 filled by KNN 

-------- Product code B ----------

filled by linear model :
measurement_17 : 418
measurement_8 : 165
measurement_11 : 220
measurement_5 : 83
measurement_6 : 106
measurement_7 : 176
measurement_4 : 80
measurement_15 : 294
measurement_10 : 197
measurement_16 : 358
measurement_14 : 330

2427 filled by linear model 
1548 filled by KNN 

-------- Product code C ----------

filled by linear model :
measurement_17 : 391
measurement_8 : 211
measurement_11 : 231
measurement_5 : 141
measurement_6 : 150
measurement_7 : 140
measurement_4 : 110
measurement_15 : 319
measurement_10 : 262
measurement_16 : 343
measurement_14 : 340

2638 filled by linear model 
1706 fill

# Build Model

In [ ]:
def build_model(total_col):
  model = Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(total_col, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1, activation="sigmoid") 
      ])
  
  return(model)

In [ ]:
features = ['loading', 'attribute_0', 'measurement_17', 'measurement_0', 
      'measurement_1', 'measurement_2','measurement_3','measurement_4',
      'measurement_5', 'measurement_6','measurement_7','measurement_8',
      'measurement_9','measurement_10','measurement_11', 'measurement_12',
      'measurement_13','measurement_14','measurement_15','measurement_16', 'measurement_17',
      'area', 'm3_missing', 'm5_missing', 'measurement_avg']

# Cross Validation

In [ ]:
X = ['A', 'B', 'C', 'D', 'E']

folds_dict = {}
i = 1
for j in range(5):
  for k in range(j + 1, 5):
    tmp_X = X.copy()
    tmp_X.remove(X[j])
    tmp_X.remove(X[k])
    tmpList = list()
    tmpList.append(tmp_X)
    tmpList.append([X[j], X[k]])
    folds_dict['#' + str(i)] = tmpList
    i += 1

# Training

In [ ]:
# training with 10-fold cross validation
for num in folds_dict.keys():   
  print(f'fold {num}')
  
  X_train, y_train = df_train[df_train['product_code'].isin(folds_dict[num][0])][features].values, df_train[df_train['product_code'].isin(folds_dict[num][0])]['failure'].values
  X_valid, y_valid = df_train[df_train['product_code'].isin(folds_dict[num][1])][features].values, df_train[df_train['product_code'].isin(folds_dict[num][1])]['failure'].values
  
  model = build_model(len(X_train))
  es_callbacks = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
  lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor="auc", factor=0.9, patience= 5, verbose=0, mode="max", min_delta=0.0001, cooldown=0, min_lr=0)
  
  model.compile(optimizer = tfa.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-3 ), loss="BinaryCrossentropy", metrics=["AUC"])
  model.fit(X_train, y_train, batch_size=64, epochs=200, callbacks=[es_callbacks, lr_scheduler], validation_data=(X_valid, y_valid))


fold #1
Epoch 1/200
254/254 [==============================] - 4s 10ms/step - loss: 0.6067 - auc: 0.5261 - val_loss: 0.5258 - val_auc: 0.5565 - lr: 0.0010
Epoch 2/200
254/254 [==============================] - 2s 8ms/step - loss: 0.5248 - auc: 0.5461 - val_loss: 0.5171 - val_auc: 0.5715 - lr: 0.0010
Epoch 3/200
254/254 [==============================] - 2s 9ms/step - loss: 0.5170 - auc: 0.5603 - val_loss: 0.5160 - val_auc: 0.5721 - lr: 0.0010
Epoch 4/200
254/254 [==============================] - 3s 11ms/step - loss: 0.5137 - auc: 0.5638 - val_loss: 0.5149 - val_auc: 0.5747 - lr: 0.0010
Epoch 5/200
254/254 [==============================] - 4s 17ms/step - loss: 0.5138 - auc: 0.5616 - val_loss: 0.5122 - val_auc: 0.5819 - lr: 0.0010
Epoch 6/200
254/254 [==============================] - 4s 15ms/step - loss: 0.5128 - auc: 0.5677 - val_loss: 0.5141 - val_auc: 0.5781 - lr: 0.0010
Epoch 7/200
254/254 [==============================] - 3s 11ms/step - loss: 0.5119 - auc: 0.5697 - val_loss: 0.5

'\noof_predictions = np.zeros((df_train.shape[0], 1))\ntest_predictions = np.zeros((df_test.shape[0], 1))\noof_targets = np.zeros((df_train.shape[0]))\ntf.random.set_seed(42)\n\n\nauc_score = []\ntargets = []\nfor fold in folds_dict.keys():\n    \n    print(f\'########################## {fold} ##########################\')\n    \n    x_train, y_train = df_train[df_train[\'product_code\'].isin(folds_dict[fold][0])][features].values, df_train[df_train[\'product_code\'].isin(folds_dict[fold][0])][\'failure\'].values\n    x_valid, y_valid = df_train[df_train[\'product_code\'].isin(folds_dict[fold][1])][features].values, df_train[df_train[\'product_code\'].isin(folds_dict[fold][1])][\'failure\'].values\n    trn_index = df_train[df_train[\'product_code\'].isin(folds_dict[fold][0])][features].index\n    vld_index = df_train[df_train[\'product_code\'].isin(folds_dict[fold][1])][features].index\n    \n    model = build_model(len(x_train))\n    es_callbacks = tf.keras.callbacks.EarlyStopping(pat

In [ ]:
joblib.dump(model, 'model.joblib')
!cp model.joblib ./gdrive/MyDrive/Final/model.joblib 